<a href="https://colab.research.google.com/github/akjieettt/dri-beautiful-sisterhood-of-books/blob/main/DRIBeautifulSisterhoodOfBooks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Loading and Importing Data

In [3]:
!git clone https://github.com/akjieettt/dri-beautiful-sisterhood-of-books
%cd dri-beautiful-sisterhood-of-books/

Cloning into 'dri-beautiful-sisterhood-of-books'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 16 (delta 4), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (16/16), 85.78 KiB | 1.45 MiB/s, done.
Resolving deltas: 100% (4/4), done.
/content/dri-beautiful-sisterhood-of-books/dri-beautiful-sisterhood-of-books


In [4]:
# Import necessary libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Load the wine quality datasets
df_bsb = pd.read_csv("BSBMasterDatabase.csv")

# Printing the number of wines in each dataset
print(f"Total Number of Entries: {len(df_bsb)}")
print(f"Total Number of NaNs:")
display(df_bsb.isna().sum())

print("\nThe Beautiful Sisterhood of Books Dataset:")
display(df_bsb.head())

Total Number of Entries: 1310
Total Number of NaNs:


,0
ID,0
author_name_variants,14
Author - Standardized (LOC listing when available),14
Contributing State,58
Title - Standardized,49
Publication Info,153
GENRE(s),77
KEYWORDS,369
Link to text,300
Research Notes,1063



The Beautiful Sisterhood of Books Dataset:


,ID,author_name_variants,Author - Standardized (LOC listing when available),Contributing State,Title - Standardized,Publication Info,GENRE(s),KEYWORDS,Link to text,Research Notes,Digital Source Notes
0,1,Marion Vaughn,"Stella (Scott) Gilman, (pseudonym Marion Vaughn)",Alabama,Mothers in Council,"New York: Harper & Brothers, 1884.","Textbook, Advice Literature","Child Rearing, Domestic Science, Conduct",https://books.google.com/books?id=4fkaAAAAYAAJ&,NaN,NaN
1,2,Mary W. Loughborough,Mary Ann (Webster) Loughborough,Arkansas,My Cave Life in Vicksburg.:[sic] With Letters ...,"New York: D. Appleton & Co., 1864.",Memoir,"Life Writing, War, Politics, Region & Regionalism",https://hdl.handle.net/2027/uc2.ark:/13960/t3v...,NaN,NaN
2,3,Mary H. Field,Mary H. (Bacon) Field,California,An Arboreal Song of the Alameda (The Beautiful...,"San Francisco: The Society, 1878.","Poetry, History","Region & Regionalism, Philosophy & Religion",https://hdl.handle.net/2027/loc.ark:/13960/t5p...,NaN,NaN
3,4,"Mary O. Stanton, M.O. Stanton",Mary Olmstead Stanton,California,Chart for the Delineations of Mental and Physi...,"Philadelphia: F.A. Davis, 1890.",Textbook,"Psychology, Science & Invention, Race",NaN,This is a pamphlet that preceded longer works ...,Still searching for digitized text.
4,5,Mrs. N. Furlong,Mrs. N. Furlong,California,"Cozenza: A Tale of Italy, and Other Poems","San Francisco: B.F. Sterrett, 1880.",Poetry,Nationalism,https://books.google.com/books?id=SwkPAAAAIAAJ&,NaN,NaN


## Visualizations